In [ ]:
import feedparser
import re
import logging
from typing import List, Dict, Set
from datetime import datetime

logger = logging.getLogger(__name__)

class RSSService:
    # Список бесплатных источников
    FEED_URLS = {
        'bcs_express': 'https://bcs-express.ru/feed',
        'finam': 'https://www.finam.ru/analysis/conews/rsspoint/',
        # Можно добавить РБК или Investing, если найти их RSS
    }

    def __init__(self, known_tickers: Set[str]):
        """
        known_tickers: список тикеров (['GAZP', 'SBER'...]), 
        чтобы мы знали, что искать в тексте.
        """
        self.known_tickers = known_tickers

    def _extract_tickers(self, text: str) -> List[str]:
        """
        Простая эвристика: ищем английские слова из 3-5 букв в верхнем регистре.
        Если слово совпадает с known_tickers — бинго!
        """
        if not text:
            return []
            
        # Ищем кандидатов (слова капсом)
        candidates = re.findall(r'\b[A-Z]{3,5}\b', text)
        
        # Фильтруем: оставляем только те, что реально торгуются на MOEX
        found = list(set([c for c in candidates if c in self.known_tickers]))
        return found

    def fetch_all_news(self) -> List[Dict]:
        """Проходит по всем RSS лентам и собирает новости"""
        all_news = []

        for source_name, url in self.FEED_URLS.items():
            logger.info(f"📡 Скачиваем ленту: {source_name}...")
            try:
                feed = feedparser.parse(url)
                
                for entry in feed.entries:
                    # Собираем полный текст для анализа
                    full_text = f"{entry.title} {entry.get('summary', '')} {entry.get('description', '')}"
                    
                    # Ищем тикеры
                    tickers = self._extract_tickers(full_text)
                    
                    # Если тикеров нет - новость "общерыночная", тоже полезно, 
                    # но для начала нам интереснее конкретные компании.
                    
                    news_item = {
                        'title': entry.title,
                        'link': entry.link,
                        'published': entry.get('published', str(datetime.now())),
                        'source': source_name,
                        'tickers': tickers,
                        'summary': entry.get('summary', '')[:500] # Обрезаем слишком длинное
                    }
                    all_news.append(news_item)
                    
            except Exception as e:
                logger.error(f"⚠️ Ошибка с лентой {source_name}: {e}")
                continue
        
        logger.info(f"✅ Всего собрано новостей: {len(all_news)}")
        return all_news
